<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/02_SQL_sqlite_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [ ]:
q = '''
CREATE TABLE S (
    Matr VARCHAR(45),
    SNome VARCHAR(45),
    Citta VARCHAR(45),
    ACorso INT,
    PRIMARY KEY (Matr)
);
'''
engine.execute(q)

q = '''
CREATE TABLE D(
 CD VARCHAR(45),
 CNome VARCHAR(45),
 Citta VARCHAR(45),
 PRIMARY KEY (CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE C(
 CC VARCHAR(45),
 CNome VARCHAR(45),
 CD VARCHAR(45),
 PRIMARY KEY (CC),
 FOREIGN KEY (CD) REFERENCES D(CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE E(
 Matr VARCHAR(45),
 CC VARCHAR(45),
 DATA DATE,
 VOTO INT,
 PRIMARY KEY (Matr, CC),
 FOREIGN KEY (Matr) REFERENCES S(Matr),
 FOREIGN KEY (CC) REFERENCES C(CC)
);
'''
engine.execute(q)


q = '''
INSERT INTO S (Matr, SNome, Citta, ACorso)
VALUES
('M1','Lucia Quaranta','SA',1),
('M2','Giacomo Tedesco','PA',2),
('M3','Carla Longo','MO',1),
('M4','Ugo Rossi','MO',1),
('M5','Valeria Neri','MO',2),
('M6','Giuseppe Verdi','BO',1),
('M7','Maria Rossi',null,1);
'''
engine.execute(q)


q = '''
INSERT INTO D (CD, CNome, Citta)
VALUES
('D1','Paolo Rossi','MO'),
('D2','Maria Pastore','BO'),
('D3','Paola Caboni','FI');
'''
engine.execute(q)


q = '''
INSERT INTO C (CC,CNome, CD)
VALUES
('C1','Fisica 1','D1'),
('C2','Analisi Matematica 1','D2'),
('C3','Fisica 2','D1'),
('C4','Analisi Matematica 2','D3');
'''
engine.execute(q)


q = '''
INSERT INTO E (Matr,CC,Data,Voto)
VALUES
('M1','C1','1995-06-29',24),
('M1','C2','1996-08-09',33),
('M1','C3','1996-03-12',30),
('M2','C1','1995-06-29',28),
('M2','C2','1996-07-07',24),
('M3','C2','1996-07-07',27),
('M3','C3','1996-11-11',25),
('M4','C3','1996-11-11',33),
('M6','C2','1996-01-02',28),
('M7','C1','1995-06-29',24),
('M7','C2','1996-04-11',26),
('M7','C3','1996-06-23',27);
'''
engine.execute(q)

## JOIN operators

In [ ]:
# Patriamo dal prodotto cartesiano: 
# Cioe' la combinazione di tutte le possibili tuple delle tabelle elencate
# nella clausola FROM
q = '''
SELECT *
FROM S,D,E,C
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
q = '''
SELECT * 
FROM S,E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# Il Join, per definizione e' un "filtro" sul prodotto cartesiano
# cioe', in algebra relazionale, una selezione sul prodotto cartesiano
q = '''
SELECT *
FROM S,E
WHERE S.Matr=E.Matr
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # che equivale a:
# q = '''
# SELECT *
# FROM S JOIN E on S.Matr=E.Matr
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

# # e anche a:
# q = '''
# SELECT *
# FROM S natural JOIN E
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

In [ ]:
# Pero' possono esseci studenti che non hanno sostenuto esami.
# Ad esempio:

q = '''
SELECT *
FROM S
WHERE S.Matr not in (SELECT Matr FROM E)
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# Se volessimo anche questi studenti nell'output del join
# dobbiamo usare un LEFT join

q = '''
SELECT * FROM S LEFT JOIN E on S.Matr=E.Matr;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# E se volessi:
# Combinazioni di studenti e di docenti residenti nella stessa citta'
# inclusi gli studenti che risiedono in una citta' 
# che non ha corrispondenza nella relazione dei docenti
q = '''
SELECT *
FROM S JOIN D ON S.Citta=D.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# con anche gli studenti di citta' da cui non viene nessun docente:
q = '''
SELECT *
FROM S LEFT JOIN D ON S.Citta=D.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# Nella clausola FROM e' possibile esprimere anche piu' condizioni di join.
# 
# Ad esempio, se volessimo:
# Per ogni esame con voto superiore a 24 riportare il nome dello studente 
# e il codice del docente del corso
q = '''
SELECT S.SNome,C.CD, E.voto
FROM (S JOIN E ON (S.Matr=E.Matr))
		JOIN C ON (E.CC=C.CC)
WHERE Voto > 24;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# che equivale a:
q = '''
SELECT S.SNome,C.CD, E.voto
FROM S,E,C
WHERE S.Matr=E.Matr 
AND E.CC=C.CC
AND Voto > 24;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# Con gli outer join, certi attributi vengono duplicati (per definizione)
# Ad sempio:
q = '''
SELECT S.Matr,S.Citta,D.CD, D.Citta
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# come faccio a fare un merge e considerare solo una Citta'?
# funzione: COALESCE(...) sceglie il primo valore non nullo
q = '''
SELECT S.Matr, S.Citta, D.Citta,D.CD
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
q = '''
SELECT S.Matr, COALESCE(S.Citta, D.Citta) AS Citta, D.CD AS CodiceDocente
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

### Self-Join

In [ ]:
# Quando il join avviene tra la tabella e se stessa
# 
# Ad esempio: 
# Selezionare le coppie di studenti della stessa citta'

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
# Cosi' pero' selezionaimo anche uno studente e se stesso
# Quindi modifichiamo aggiungendo:

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta
AND S1.Matr < S2.Matr;
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

#
# DOMANDA:
# perche' <> non andrebbe bene?
#

In [ ]:
# Matricole degli studenti che hanno sostenuto almeno 
# uno degli esami sostenuti dallo studente di nome 'Giuseppe Verdi'

q = '''
SELECT E1.Matr
FROM S, E E1, E E2
WHERE E2.Matr = S.Matr
AND E1.CC = E2.CC
AND S.SNome='Giuseppe Verdi';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

In [ ]:
q = '''
SELECT E1.Matr
FROM E E1
WHERE E1.CC IN (SELECT E2.CC
				FROM E E2, S
				WHERE S.SNome='Giuseppe Verdi'
				AND S.Matr = E2.Matr);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df